We can make a slight improvement to the **Hunt** part of the hunt and target algorithm using parity.


Because the minimum length of a ship is two units long, then we
don't need to random search every location on the board. Even
the shortest ship has to straddle two adjacent squares.

A mathematical term to describe this is **Parity**. This is just a
fancy word of describing if the square would contain an odd or
even number if labelled sequentially from 1 to 100.

The blue squares on the grid are even parity and the white
squares odd parity.

![Parity ](https://i.imgur.com/g5zbP0t.png)

We can instruct our **Hunt** algorithm to only randomly fire into
unknown locations with even parity. Even if we only ever fire at
blue locations, we will at least hit every ship — it's impossible to
place any ship so that it does not touch at least one blue
square.

Once a target as been hit, and Target mode is activated, the
'parity' restriction is lifted enabling all potential targets to be
investigated. If the algorithm returns to Hunt mode, again, the
parity filter is enabled.

Some readers will have realised that, once we've sunk
the two unit destroyer, we can change the parity restriction to a
larger spacing, but we're not going to implement that bit.

In [5]:
# TODO: put some more cells in here

The parity algorithm gives improvment over the entire range, but the incremental
gain is small. The biggest drain on shots is the unecessary walking around the edges of targets. Using the parity filter in Hunt mode has
reduced the shot count, but once the algorithm gets into Target mode, it is just as inefficient as it was. To make futher improvements in
strategy, it is this area we need to focus our attention.